In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
# print(GOOGLE_API_KEY)
LANGCHAIN_API_KEY=os.getenv("LANGSMITH_API_KEY")

LANGCHAIN_PROJECT=os.getenv("LANGSMITH_PROJECT")
LANGCHAIN_TRACING_V2 = os.getenv("LANGSMITH_TRACING")
LANGCHAIN_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

#### ALternative

In [6]:
# '''from langchain_huggingface import HuggingFaceEmbeddings
# embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
# from langchain_groq import ChatGroq
# import os
# llm=ChatGroq(model_name="Gemma2-9b-It")

In [7]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hi there! How can I help you today?
goodbye take care yourself


#### memory Concept

In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store = {}

In [10]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
config = {"configurable": {"session_id": "firstchat"}}

In [12]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [13]:
model_with_memory.invoke(("Hi! I'm Pavan Manikanta"),config=config).content

"Hi Pavan Manikanta! It's nice to meet you.  Is there anything I can help you with today?"

In [14]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Pavan Manikanta.  You told me at the beginning of our conversation.'

In [15]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Pavan Manikanta", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Pavan Manikanta! It's nice to meet you.  Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-c2c30268-275d-4dd0-9a57-537cc4479b2f-0', usage_metadata={'input_tokens': 10, 'output_tokens': 27, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Pavan Manikanta.  You told me at the beginning of our conversation.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='ru

#### Implementing LCEL

In [16]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [17]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

In [18]:
docs

[Document(metadata={'source': '../data/firebase_studio.txt'}, page_content='Firebase Studio: An AI-Powered Cloud IDE for Seamless Full-Stack Development\n\n1. Introduction: The Evolving Landscape of Application Development and the Advent of Cloud IDEs\n\nThe world of software development is in a constant state of flux, driven by ever-increasing demands for more complex, feature-rich, and rapidly delivered applications. From simple websites, we\'ve moved to dynamic web applications, powerful mobile apps, and intricate systems that connect users across multiple devices and platforms. This evolution has led to a significant rise in the prominence of "full-stack" development, where developers are expected to possess skills across the entire application stack, from the user interface (frontend) to the server-side logic (backend), databases, and deployment infrastructure.\n\nThis growing complexity places considerable demands on developers and their tools. Setting up and managing local devel

In [19]:
### Creating Chunks using RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 120,
    chunk_overlap = 20,
    length_function=len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

In [20]:
### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [21]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [22]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [24]:
question ="Is it a replacement for local IDEs?"
print(retrieval_chain.invoke(question))

It doesn't necessarily replace traditional local development environments for all scenarios.


#### Start with Tools and Agents

In [25]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [52]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5 )
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool.name

'wikipedia'

In [53]:
wiki_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [28]:
wiki_tool.run("langcgraph")

'No good Wikipedia Search Result was found'

In [29]:
from langchain_core.pydantic_v1 import BaseModel, Field

/home/pavan/Ds/pro/langgraph/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3667: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:

from langchain_community.tools import YouTubeSearchTool

In [31]:
utube_tool=YouTubeSearchTool()
utube_tool.name

'youtube_search'

In [32]:
utube_tool.run("machine learning")

"['https://www.youtube.com/watch?v=ukzFI9rgwfU&pp=ygUQbWFjaGluZSBsZWFybmluZw%3D%3D', 'https://www.youtube.com/watch?v=PcbuKRNtCUc&pp=ygUQbWFjaGluZSBsZWFybmluZw%3D%3D']"

In [33]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()

In [34]:
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'title': 'What Happened At Burning Man 2023? Death, Flooding And Sticky ...',
  'url': 'https://www.marieclaire.com.au/latest-news/burning-man-festival-2023-explainer/',
  'content': 'More than 70,000 festival attendees have been left stranded in the desert after a heavy storm flooded the grounds. Images show the usually colourful, art-filled environment\xa0as a wasteland, strewn\xa0with rubbish, flooded tents and ankle-deep mounds of ‘sticky mud’.\n\nBut that’s not all—authorities also say they’re investigating a death that’s occurred at the event.\n\nSo what happened? We explain below.\n\nNative ad body. [...] The festival disaster started with a storm—and not just any storm. The remote area was hit with two to three months worth of rain in just 24 hours. When the heavy rain hit the dry desert earth, it created a thick, ‘sticky’ mud with a clay like consistency\xa0that’s difficult to walk through.\n\nAttendees were told to shelter in place and to conserve food, water and fuel until

agents

In [35]:
from langchain import hub
from langchain.agents import AgentExecutor,create_openai_functions_agent

In [36]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [37]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instructions'], input_types={}, partial_variables={}, template='{instructions}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [38]:
tavily_tool = TavilySearchResults()

In [54]:
tools = [tavily_tool,utube_tool,wiki_tool]

In [55]:
# agent = create_openai_functions_agent(llm, tools, prompt)
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [56]:

agent_executor = AgentExecutor(
  agent=agent,
  tools=tools,
  verbose=True,
)



In [57]:
print(agent_executor.invoke({"input": "who was the Veer Savarkar"}))





> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Veer Savarkar'}`


Page: Vinayak Damodar Savarkar
Summary: Vinayak Damodar Savarkar (28 May 1883 – 26 February 1966)  was an Indian politician, activist and writer. Savarkar developed the Hindu nationalist political ideology of Hindutva while confined at Ratnagiri in 1922. He was a leading figure in the Hindu Mahasabha. The prefix "Veer" (meaning 'brave') has been given by himself, when he penned his own biography under pseudonym of Chitragupta .
Savarkar began his political activities as a high school student and continued to do so at Fergusson College in Pune. He and his brother founded a secret society called Abhinav Bharat Society. When he went to the United Kingdom for his law studies, he involved himself with organizations such as India House and the Free India Society. He also published books advocating complete Indian independence by revolutionary means. One of the books he published called The Indi